# Parallelism

## Threads

**Multithreading** - a way to achieve concurrency within a single process

Multithreading refers to a process where a single process creates multiple threads of execution that run concurrently. The threads share the same memory space, so they can communicate and access the same data. This makes it easier to write concurrent programs, as the threads can communicate and synchronize with each other easily. However, multithreading has some drawbacks, including the potential for race conditions and other concurrency-related bugs.

Since I/O-bound tasks involve waiting for data to be read from or written to a resource, the Python interpreter can switch between threads while waiting for I/O operations to complete, allowing the CPU to work on other tasks in the meantime. This can result in better overall performance and faster completion times for the program.

## Processes

**MultiProcessing** - a way to achieve parallelism across multiple processes

Multiprocessing, on the other hand, refers to a process where multiple independent processes are running in parallel. Each process has its own memory space, so they cannot communicate or access the same data without explicit communication mechanisms like pipes or message queues. Multiprocessing can be more efficient for tasks that are CPU-bound, as it can take advantage of multiple CPU cores to execute tasks in parallel. However, it can be more difficult to write and debug multiprocessing code, since the processes do not share memory and require explicit communication mechanisms.



## Semaphore Data Structure

Semaphores are a good way to keep threads from hitting race conditions (overlapping each other) by acquiring locks. 

The concept is simple, 
- semaphore is initialized to a variable, let's call it **counter**
- acquire() = -1 to the counter
- release() = +1 to the counter
- if the counter = 0, an attempt at an acquire() will be blocked until a different thread calls the release() method on the semaphore

In [1]:
import threading

class Semaphore():
    def __init__(self, max_available):
        self.cv = threading.Condition()
        self.MAX_AVAILABLE = max_available
        self.taken = 0

    def acquire(self):
        self.cv.acquire() # Acquire thread
        while (self.taken == self.MAX_AVAILABLE):
            self.cv.wait()
        self.taken += 1
        self.cv.release() # Release thread
    
    def release(self):
        self.cv.acquire() # Acquire Thread
        self.taken -= 1
        self.cv.notify()
        self.cv.release() # Release Thread

## Example Usage

In [2]:
import time

class Resource():
    def __init__(self):
        self.semaphore = Semaphore(2) # Modify the integer value to see interesting results and make things click!! :) 

    def access(self):
        self.semaphore.acquire()
        print("Accessing resource...")
        time.sleep(1)
        print("Done accessing resource.")
        self.semaphore.release()

In [3]:
# create a shared resource object
resource = Resource()

# define a function that accesses the resource
def worker():
    resource.access()

# create several threads that will run the worker function
threads = [threading.Thread(target=worker) for i in range(10)]

# start the threads
for t in threads:
    t.start()

# wait for the threads to finish
for t in threads:
    t.join()

Accessing resource...
Accessing resource...
Done accessing resource.
Done accessing resource.
Accessing resource...
Accessing resource...
Done accessing resource.Done accessing resource.
Accessing resource...

Accessing resource...
Done accessing resource.
Done accessing resource.
Accessing resource...
Accessing resource...
Done accessing resource.
Done accessing resource.
Accessing resource...
Accessing resource...
Done accessing resource.
Done accessing resource.


## Tips

Using a `with Semaphore` will auto acquire the lock at the beginning and release it when done so you don't have to be explicit about it